In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'E:\DataSets\amazon500k\Reviews.csv')

In [3]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df["Score"].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [5]:
df_neg = df.loc[df["Score"] < 3]
df_neg = df_neg.reset_index(drop = True)

In [6]:
df_five = df.loc[df["Score"] == 5]
df_five = df_five.reset_index(drop = True)

In [7]:
print(len(df_neg))
print(len(df_five))

82037
363122


In [8]:
df_pos = df_five.loc[:len(df_neg)]

In [9]:
df_all = pd.concat([df_neg, df_pos], axis =0 )
df_all = df_all.reset_index(drop = True)

In [10]:
df_all["Sentiment"]=np.where(df_all["Score"]==5,"Positive","Negative")

In [11]:
df_all = df_all.sample(frac = 1)
df_all = df_all.reset_index(drop = True)

In [12]:
df_all.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Sentiment
0,83379,B005ZBZLT4,A26NJ6PTJVP79W,cl,0,0,5,1349568000,coffee,I love this coffee. I love getting a value on ...,Positive


In [13]:
df_all.drop(df_all.columns[[0,1,2,3, 4,5,7,8]], axis=1, inplace=True)


In [14]:
df_all.head()

,Score,Text,Sentiment
0,5,I love this coffee. I love getting a value on ...,Positive
1,5,I like hot chocolate and I drink a lot of it. ...,Positive
2,5,This is the only brand which I could find that...,Positive
3,5,I like this product. It is a fast and easy wa...,Positive
4,1,We just made our first loaf in our 2 lb breadm...,Negative


In [25]:
from gensim.parsing.preprocessing import remove_stopwords
df_all['Text'] = df_all['Text'].apply(remove_stopwords)

In [26]:
df_all.head()

,Score,Text,Sentiment
0,5,I love coffee. I love getting value it. I use ...,Positive
1,5,I like hot chocolate I drink lot it. Naturally...,Positive
2,5,This brand I sells decaf chai loose leaf tea. ...,Positive
3,5,I like product. It fast easy way garlic bread....,Positive
4,1,We loaf 2 lb breadmaker (which usually produce...,Negative


In [16]:
len(df_all)

200

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_all.Text,df_all.Sentiment)


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
x_train_vec=v.fit_transform(x_train)
x_test_vec = v.transform(x_test)

In [19]:
from sklearn import svm
clf_svm = svm.SVC(kernel="linear")
clf_svm.fit(x_train_vec,y_train)

SVC(kernel='linear')

In [20]:
clf_svm.score(x_test_vec,y_test)

0.7

In [21]:
rev = ["This product is amazing"]
rev_vec = v.transform(rev)
clf_svm.predict(rev_vec)

array(['Positive'], dtype=object)

In [22]:
rev = [""]
rev_vec = v.transform(rev)
clf_svm.predict(rev_vec)

array(['Positive'], dtype=object)